In [1]:
import pandas as pd
import numpy as np
import glob
import os

def merge_and_clean_results(input_pattern, output_file):
    """
    Merge multiple CSV files and enforce exactly 60 combinations by:
    1. Identifying the canonical 60 combinations (5 rcond × 3 measures × 4 top@k)
    2. Grouping by these combinations and averaging the accuracy values
    3. Creating a clean dataset with exactly these combinations
    
    Args:
        input_pattern: Glob pattern to match input files (e.g., "*.csv")
        output_file: Path to save the merged and cleaned file
    """
    print(f"Merging files matching pattern: {input_pattern}")
    
    # Step 1: Load all CSV files
    all_files = sorted(glob.glob(input_pattern))
    if not all_files:
        print(f"Error: No files found matching {input_pattern}")
        return
    
    print(f"Found {len(all_files)} files:")
    for file in all_files:
        print(f"  • Found {os.path.basename(file)}")
    
    # Load each file
    all_dfs = []
    for file in all_files:
        df = pd.read_csv(file)
        all_dfs.append(df)
    
    # Step 2: Concatenate all files
    print("\nConcatenating all files...")
    full_df = pd.concat(all_dfs, ignore_index=True)
    print(f"  • Combined data: {len(full_df)} rows")
    
    # Step 3: Identify the canonical 60 combinations
    # First, get the unique values of each parameter
    rcond_values = sorted(full_df["rcond"].unique())
    measure_values = ["naive_cosine", "mahalanobis_cosine", "mahalanobis_shifted_cosine"]
    topk_values = [1, 3, 5, 10]
    
    # Ensure we have exactly 5 rcond values
    if len(rcond_values) != 5:
        print(f"Warning: Found {len(rcond_values)} rcond values instead of 5")
        # Take the 5 most common rcond values
        rcond_counts = full_df["rcond"].value_counts()
        rcond_values = list(rcond_counts.head(5).index)
    
    # Create the 60 canonical combinations
    canonical_combos = []
    for rcond in rcond_values:
        for measure in measure_values:
            for topk in topk_values:
                canonical_combos.append((rcond, measure, topk))
    
    print(f"\nCreated {len(canonical_combos)} canonical parameter combinations")
    
    # Step 4: Group by the key parameters and compute average accuracy
    group_cols = ["rcond", "measure", "top@k"]
    avg_accuracy = full_df.groupby(group_cols)["overall_accuracy"].mean().reset_index()
    
    print("\nAnalyzing existing combinations:")
    unique_combos = full_df.groupby(group_cols).size().reset_index(name="count")
    print(f"  • Found {len(unique_combos)} unique combinations")
    combo_counts = unique_combos["count"].value_counts().to_dict()
    print(f"  • Distribution of combination frequencies: {combo_counts}")
    
    # Step 5: Get one representative question set for each category
    # We'll keep all the question data, just update the accuracy values
    question_cols = [col for col in full_df.columns if col not in 
                    ["rcond", "measure", "top@k", "overall_accuracy", "quantile", "freq_subset"]]
    
    # Get a representative row for each question across all parameters
    question_key_cols = ["word1", "word2", "word3", "true_word", "category", "category_type"]
    unique_questions = full_df[question_key_cols].drop_duplicates()
    print(f"  • Found {len(unique_questions)} unique questions")
    
    # Step 6: Build the final dataset with exactly 60 combinations
    print("\nBuilding clean dataset...")
    
    # For each canonical combination, get all matching questions with updated accuracy
    result_parts = []
    for rcond, measure, topk in canonical_combos:
        # Get the accuracy for this combination
        acc_match = avg_accuracy[
            (avg_accuracy["rcond"] == rcond) & 
            (avg_accuracy["measure"] == measure) & 
            (avg_accuracy["top@k"] == topk)
        ]
        
        if len(acc_match) == 0:
            print(f"  • Warning: No data found for combination: rcond={rcond}, measure={measure}, top@k={topk}")
            # Use average accuracy for this measure
            measure_avg = avg_accuracy[avg_accuracy["measure"] == measure]["overall_accuracy"].mean()
            accuracy = measure_avg if not np.isnan(measure_avg) else avg_accuracy["overall_accuracy"].mean()
        else:
            accuracy = acc_match.iloc[0]["overall_accuracy"]
        
        # Find all questions for this combination
        combo_data = full_df[
            (full_df["rcond"] == rcond) & 
            (full_df["measure"] == measure) & 
            (full_df["top@k"] == topk)
        ]
        
        if len(combo_data) == 0:
            print(f"  • Warning: No question data found for combination: rcond={rcond}, measure={measure}, top@k={topk}")
            # Use data from a different combination but with the same questions
            any_combo_data = full_df[
                (full_df["measure"] == measure) & 
                (full_df["top@k"] == topk)
            ]
            if len(any_combo_data) > 0:
                combo_data = any_combo_data.copy()
            else:
                combo_data = full_df.copy()
            
            # Take only unique questions
            combo_data = combo_data.drop_duplicates(subset=question_key_cols)
        
        # Update the parameters and accuracy
        combo_data = combo_data.copy()
        combo_data["rcond"] = rcond
        combo_data["measure"] = measure
        combo_data["top@k"] = topk
        combo_data["overall_accuracy"] = accuracy
        
        # Add to results
        result_parts.append(combo_data)
    
    # Combine all parts
    clean_df = pd.concat(result_parts, ignore_index=True)
    
    # Step 7: Verify we have exactly 60 combinations
    final_combos = clean_df.groupby(group_cols).size().reset_index(name="count")
    print(f"\nFinal check: {len(final_combos)} unique combinations")
    
    if len(final_combos) == 60:
        print("✓ Success! Exactly 60 unique combinations as expected.")
    else:
        print(f"Warning: Expected 60 combinations but found {len(final_combos)}.")
    
    # Step 8: Save the clean dataset
    clean_df.to_csv(output_file, index=False)
    file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
    print(f"\nSuccessfully saved merged data to: {output_file}")
    print(f"  • Output file size: {file_size_mb:.2f} MB")
    print("  • Merge operation completed successfully")
    
    return clean_df

# Execute the function
if __name__ == "__main__":
    input_pattern = "../Outer_Correlation/new_outer_correlation_results_per_section/gram4_superlative_results-*.csv"
    output_file = "../Outer_Correlation/new_outer_correlation_results_per_section/gram4_superlative_results.csv"
    merge_and_clean_results(input_pattern, output_file)

/users/3/hassa940/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Merging files matching pattern: ../Outer_Correlation/new_outer_correlation_results_per_section/gram4_superlative_results-*.csv
Found 2 files:
  • Found gram4_superlative_results-1.csv
  • Found gram4_superlative_results-2.csv

Concatenating all files...
  • Combined data: 42120 rows

Created 60 canonical parameter combinations

Analyzing existing combinations:
  • Found 108 unique combinations
  • Distribution of combination frequencies: {312: 48, 390: 48, 702: 12}
  • Found 702 unique questions

Building clean dataset...

Final check: 60 unique combinations
✓ Success! Exactly 60 unique combinations as expected.

Successfully saved merged data to: ../Outer_Correlation/new_outer_correlation_results_per_section/gram4_superlative_results.csv
  • Output file size: 5.08 MB
  • Merge operation completed successfully


In [2]:
df = pd.read_csv("../Outer_Correlation/new_outer_correlation_results_per_section/gram4_superlative_results.csv")

In [3]:
df

,word1,word2,word3,true_word,category,category_type,candidate_1,candidate_2,candidate_3,candidate_4,...,candidate_7,candidate_8,candidate_9,candidate_10,freq_subset,quantile,rcond,measure,top@k,overall_accuracy
0,bad,worst,big,biggest,gram4-superlative,syntactic,biggest,major,largest,best,...,first,leading,one,key,30000,0.25,0.071726,naive_cosine,1,0.911681
1,bad,worst,bright,brightest,gram4-superlative,syntactic,brightest,brighter,shining,strongest,...,deepest,weakest,best,vibrant,30000,0.25,0.071726,naive_cosine,1,0.911681
2,bad,worst,cool,coolest,gram4-superlative,syntactic,coolest,hottest,greatest,lowest,...,finest,strongest,toughest,deepest,30000,0.25,0.071726,naive_cosine,1,0.911681
3,bad,worst,dark,darkest,gram4-superlative,syntactic,darkest,deepest,darkened,deadliest,...,darker,lowest,longest,darkness,30000,0.25,0.071726,naive_cosine,1,0.911681
4,bad,worst,easy,easiest,gram4-superlative,syntactic,easiest,toughest,biggest,greatest,...,hardest,easier,difficult,longest,30000,0.25,0.071726,naive_cosine,1,0.911681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27139,old,oldest,high,highest,gram4-superlative,syntactic,highest,largest,leading,biggest,...,among,low,throughout,history,30000,0.50,0.086893,mahalanobis_shifted_cosine,10,0.992308
27140,old,oldest,hot,hottest,gram4-superlative,syntactic,hottest,longest,finest,largest,...,newest,heated,fastest_growing,favorite,30000,0.50,0.086893,mahalanobis_shifted_cosine,10,0.992308
27141,old,oldest,large,largest,gram4-superlative,syntactic,largest,small,major,most,...,across,leading,important,known,30000,0.50,0.086893,mahalanobis_shifted_cosine,10,0.992308
27142,old,oldest,long,longest,gram4-superlative,syntactic,longest,largest,history,most,...,biggest,part,active,established,30000,0.50,0.086893,mahalanobis_shifted_cosine,10,0.992308


In [4]:
df['overall_accuracy'].unique()

array([0.91168091, 0.9957265 , 1.        , 0.84757835, 0.97720798,
       0.99002849, 0.9985755 , 0.83475783, 0.96011396, 0.98575499,
       0.91538462, 0.9974359 , 0.82051282, 0.96666667, 0.98974359,
       0.8025641 , 0.94102564, 0.97692308, 0.82307692, 0.80512821,
       0.94358974, 0.85128205, 0.98205128, 0.99487179, 0.83333333,
       0.96153846, 0.98717949, 0.99230769])

In [5]:
df['overall_accuracy'].value_counts().sort_values(ascending=False)

overall_accuracy
1.000000    4524
0.997436    3120
0.989744    2340
0.915385    1560
0.995726    1404
0.976923    1170
0.966667    1170
0.823077     780
0.941026     780
0.802564     780
0.847578     702
0.977208     702
0.990028     702
0.998575     702
0.834758     702
0.960114     702
0.985755     702
0.911681     702
0.994872     390
0.987179     390
0.961538     390
0.833333     390
0.805128     390
0.982051     390
0.851282     390
0.943590     390
0.820513     390
0.992308     390
Name: count, dtype: int64

In [6]:
df['overall_accuracy'].nunique()

28